In [13]:
import codecs
import io, json, codecs
import os
from bs4 import BeautifulSoup as bs
import fix

In [14]:
def oai_json(journal_name,oai_file):
    soup=bs(xml_content,"lxml").findAll('metadata')
    global articles; articles==[]
    global e
    for s in soup:
        creators=[] 
        article={}
        article['ex:creator']=[]
        article['ex:keyword']=[]
        properties=s.findChild().findAll()
        for p in properties:
            key= p.name
            j=p.text.strip()
            if key =='dc:title':article['ex:title']=j
            elif key=='dc:date':article['ex:date']=j[0:4]
            elif key=='dc:identifier':
                if 'vol' in j:
                    if 'iss' in j:
                        issue=j.split('iss')[1].split('/')[0]
                        article['ex:issue']=issue                        
                    volume=j.split('vol')[1].split('/')[0]
                    article['ex:volume']=volume
                    article['ex:url']=j
            elif key=='dc:description':article['ex:abstract']=j
            elif key=='dc:source':article['ex:source']=j
            elif key=='dc:creator':
                creator={}
                if ',' in j:
                    j=' '.join(reversed([part.strip() for part in j.split(',')]))
                creator['ex:name']=j
                creators.append(creator)
            elif key=='dc:subject': article['ex:keyword'].append(fix.fix_keyword(j))
        article['ex:creator']=creators
        if fix.validate(article)==True:
            articles.append(article)
        else:
            e=e+1
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_folder+oai_file.replace('.xml','.jsonld').replace('/','_')
    with io.open(jsonld_file, 'w') as f:
        f.write(jd)
    return (jsonld_file)

In [15]:
with open('journals.json') as data_file:    
    journals = json.load(data_file)

oai_folder='oai/'
jsonld_folder='jsonld/'

for journal in journals:
    source=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='oai':
            print publisher,source,journal_name,
            i=0
            e=0
            for file_name in os.listdir(oai_folder+publisher+'/'+source):
                if file_name[-4:]=='.xml':
                    oai_file=publisher+'/'+source+'/'+file_name
                    content = []
                    jd=''
                    articles=[]
                    with codecs.open(oai_folder+oai_file) as myfile:
                        xml_content=myfile.read().replace('\n', '')
                    jsonld_file= oai_json(journal_name,oai_file)
                i=i+1
            print 'fixes=',e,' files=',i

dgc misqrt Management Information Systems Quarterly fixes= 0  files= 15


In [16]:
jsonld_file

u'jsonld/dgc_misqrt_oai9.jsonld'